In [1]:
#Test unravel and also upsampling method!
import tensorflow as tf
import numpy as np


def max_pool(inputs,name):
    value,index = tf.nn.max_pool_with_argmax(tf.to_double(inputs),ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name=name)
    print('value shape',value.shape)
    print('index shape',index.shape)
    
    return value,index



def unravel_index(indices, shape):
    with tf.name_scope('unravel_index'):
        indices = tf.to_int64(tf.expand_dims(indices, 0))
        shape = tf.to_int64(tf.expand_dims(shape, 1))
        strides = tf.to_int64(tf.cumprod(shape, reverse=True))
        strides_shifted = tf.to_int64(tf.cumprod(shape, exclusive=True, reverse=True))
        return (indices % strides) // strides_shifted
    #This function is utilized to transform the flattened maxpooling index to the original 4D tensor, and have already test
    #it, which works brilliant!
    """
    indices: indices will be the output index from maxpooling, just to make sure if it's only 1D!
    Shape: the shape of the original data,[batch_size,height,width,Num_of_Channels]
    output: It's the 4D maxpooling indices!
    """
    
def up_sampling(value,indices,shape):
    """
    Inputs:
    value: the maximum value from maxpooling function, value need to be a tensor. The most important thing for
    value is that it needs to be reshaped to be only one column! 
    indices: the flattened position for the maximum value from maxpooling function. Also indices need to be reshaped
    to be two dimension. [Num_tot_values,4]. 4 is because we have 4 dimension
    shape: the shape of the original data, [batch_size,height,width,Num_of_Channels]
    Outputs:
    up_sample_sp: The sparse matrix from the up_sampling.
    """
    tot = np.prod(value.shape) #product of all sizes
    value_reshape = tf.reshape(value,[tot])
    index_reshape = tf.reshape(indices,[1,tot])
    print('The shape of reshaped maxvalue',value_reshape.shape)
    print('The shape of reshaped maxindex',index_reshape.shape)
    pooling_index_4d = unravel_index(index_reshape,shape)
    print('The shape of 4d indices', pooling_index_4d.shape)
    s = tf.Session()
    sp_tensor = tf.SparseTensor(tf.reshape(pooling_index_4d,
                                           [pooling_index_4d.shape[2].value,
                                            pooling_index_4d.shape[1].value]),
                                            values = value_reshape, dense_shape = shape)
    sp_dense = tf.sparse_tensor_to_dense(sp_tensor)
    print('The sp_dense?',s.run(sp_dense))
    print('The shape of sparse matrix', sp_dense.shape)
    return s.run(sp_dense) 


In [8]:
# let's assume that inputs is a matrix which is 4D matrix, the dimension is (Num_Batch, width, height, Num_channel)
shape = [1,5,5,2]
temp = np.array([[[[1,0],[2,0],[3,0],[400,0],[5,0]],[[600,0],[7,0],[8,0],[9,0],[1000,0]],[[11,0],[1200,0],[1300,0],[14,0],[15,0]],[[16,0],[17,0],[18,0],[19,0],[2000,0]],[[21,0],[2200,0],[2300,0],[24,0],[2500,0]]]])
print(temp)
max_values, max_indices = max_pool(temp,name='PLAY')
print(max_values)
print(max_indices)

[[[[   1    0]
   [   2    0]
   [   3    0]
   [ 400    0]
   [   5    0]]

  [[ 600    0]
   [   7    0]
   [   8    0]
   [   9    0]
   [1000    0]]

  [[  11    0]
   [1200    0]
   [1300    0]
   [  14    0]
   [  15    0]]

  [[  16    0]
   [  17    0]
   [  18    0]
   [  19    0]
   [2000    0]]

  [[  21    0]
   [2200    0]
   [2300    0]
   [  24    0]
   [2500    0]]]]
value shape (1, 3, 3, 2)
index shape (1, 3, 3, 2)
Tensor("PLAY_2:0", shape=(1, 3, 3, 2), dtype=float64)
Tensor("PLAY_2:1", shape=(1, 3, 3, 2), dtype=int64)


In [11]:
values_reshape = tf.reshape(max_values,[-1])
indices_reshape = tf.reshape(max_indices,[-1])
print('The shape of reshaped maxvalue',values_reshape.shape)
print('The shape of reshaped maxindex',indices_reshape.shape)
pooling_index_4d = tf.stack(tf.unstack(unravel_index(indices_reshape,shape), axis=0), axis=1)
print('The shape of 4d indices', pooling_index_4d.shape)
with tf.Session() as sess:
    sp_tensor = tf.SparseTensor(pooling_index_4d, values = values_reshape, dense_shape = shape)
    #sp_dense = tf.sparse_tensor_to_dense(sp_tensor)
    #sess.run(sp_dense)
    print(sess.run(sp_tensor))

The shape of reshaped maxvalue (18,)
The shape of reshaped maxindex (18,)
The shape of 4d indices (18, 4)
SparseTensorValue(indices=array([[0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 3, 0],
       [0, 0, 2, 1],
       [0, 1, 4, 0],
       [0, 0, 4, 1],
       [0, 2, 1, 0],
       [0, 2, 0, 1],
       [0, 2, 2, 0],
       [0, 2, 2, 1],
       [0, 3, 4, 0],
       [0, 2, 4, 1],
       [0, 4, 1, 0],
       [0, 4, 0, 1],
       [0, 4, 2, 0],
       [0, 4, 2, 1],
       [0, 4, 4, 0],
       [0, 4, 4, 1]]), values=array([  600.,     0.,   400.,     0.,  1000.,     0.,  1200.,     0.,
        1300.,     0.,  2000.,     0.,  2200.,     0.,  2300.,     0.,
        2500.,     0.]), dense_shape=array([1, 5, 5, 2]))


In [4]:
s = tf.Session()
out = unravel_index([4, 6, 10, 12, 13, 20, 22, 23, 25], (5, 5))
print(out)
print(s.run(out))

Tensor("unravel_index_1/floordiv:0", shape=(2, 9), dtype=int64)
[[0 1 2 2 2 4 4 4 0]
 [4 1 0 2 3 0 2 3 0]]
